In [1]:
import json
import fsspec

from query_insights.utils import read_and_process_data

In [ ]:
data_dict_table_path = "../data/data_dictionary/raw/"
output_json_file_path = "../data/data_dictionary/"

# Give the list of input files (csv/xlsx) that contains two columns [column_name, column_description]. JSON file names will be same as the input file names
file_names = ["invoice_data.csv"]

In [2]:

def json_file_generator(
    data_dict_table_path,
    file_name,
    output_json_file_path,
    fs=None,
    **kwargs,
):
    """
    generate a json file named `<file_name>.json` from a data dictionary file

    Parameters
    ----------
    data_dict_table_path : str
        input folder path containing the data dictionary (csv/xlsx) file
    file_name : str
        name of the data dictionary (csv/xlsx) file.
    output_json_file_path : str
        output folder path where the JSON files will be saved.
    fs : fsspec.filesystem, optional
        Filesystem of the url, by default ``None``

    Returns
    -------
    None
    """
    fs = fs or fsspec.filesystem("file")
    out_dict = {}
    ls = []
    df = read_and_process_data(pp.join(data_dict_table_path, file_name))[["column_name", "column_description"]]
    
    pattern = re.compile(r"[^\w]")
    
    df['column_name'] = [col.lower().replace(" ", "_") for col in df['column_name']]
    df['column_name'] = [pattern.sub("_", col) for col in df['column_name']]

    mapp = dict(df.values)

    for key, val in mapp.items():
        ls.append({"column_name": key, "column_description": val})
    out_dict["table_name"] = file_name.split(".")[0]
    out_dict["columns"] = ls
    with fs.open(pp.join(output_json_file_path, f"{out_dict['table_name']}.json"), "w") as f:
        json.dump(out_dict, f, indent=4, **kwargs)


In [ ]:
for file_name in file_names:
    json_file_generator(data_dict_table_path, file_name, output_json_file_path)